In [1]:
!pip install beautifulsoup4
#!wget https://dumps.wikimedia.org/enwiki/20190420/enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2
#!bunzip2 enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2    

In [2]:
!pip install lxml

In [3]:
# Count lines, words, characters in XML data file
!wc enwiki-20190420-pages-articles-multistream1.xml-p10p30302

 4586335 74563548 642819893 enwiki-20190420-pages-articles-multistream1.xml-p10p30302


In [4]:
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
def get_pages_from_xml_file(filename, start_tag='<page>', end_tag='</page>'):
    """Yields each page from the specified XML data file."""
    page = None
    with open(filename) as f:
        for line in f:
            if start_tag in line:
                page = []
                page.append(line)
            elif end_tag in line:
                page.append(line)
                page_xml = ''.join(page)
                yield page_xml
                page = None
            else:
                if page is not None:
                    page.append(line)

In [6]:
filename = 'enwiki-20190420-pages-articles-multistream1.xml-p10p30302'
pages = list(get_pages_from_xml_file(filename))

In [7]:
def get_title_from_page_xml(page_xml):
    """Returns the title of the given page."""
    soup = BeautifulSoup(page_xml, 'lxml')
    return soup.select_one('title').text

In [8]:
len(pages)  # Show the number of pages

19822

In [9]:
%time titles = [get_title_from_page_xml(page) for page in pages]

CPU times: user 32 s, sys: 1.27 s, total: 33.3 s
Wall time: 33.3 s


In [10]:
titles[:5] + ['...'] + titles[-5:]  # Show the first and last 5 titles

['AccessibleComputing',
 'Anarchism',
 'AfghanistanHistory',
 'AfghanistanGeography',
 'AfghanistanPeople',
 '...',
 'The Lord of the Rings/One Ring',
 'Tax Freedom Day',
 'Tax',
 'Transhumanism',
 'TARDIS']

In [11]:
print(pages[0])  # Print the first page

  <page>
    <title>AccessibleComputing</title>
    <ns>0</ns>
    <id>10</id>
    <redirect title="Computer accessibility" />
    <revision>
      <id>854851586</id>
      <parentid>834079434</parentid>
      <timestamp>2018-08-14T06:47:24Z</timestamp>
      <contributor>
        <username>Godsy</username>
        <id>23257138</id>
      </contributor>
      <comment>remove from category for seeking instructions on rcats</comment>
      <model>wikitext</model>
      <format>text/x-wiki</format>
      <text xml:space="preserve">#REDIRECT [[Computer accessibility]]

{{R from move}}
{{R from CamelCase}}
{{R unprintworthy}}</text>
      <sha1>42l0cvblwtb4nnupxm6wo000d27t6kf</sha1>
    </revision>
  </page>



In [12]:
def get_text_from_page_xml(page_xml):
    """Returns the text of the given page."""
    soup = BeautifulSoup(page_xml, 'lxml')
    return soup.get_text()

In [13]:
page = pages[0]
soup = BeautifulSoup(page, 'lxml')
text = get_text_from_page_xml(page)
text, type(text)

('\nAccessibleComputing\n0\n10\n\n\n854851586\n834079434\n2018-08-14T06:47:24Z\n\nGodsy\n23257138\n\nremove from category for seeking instructions on rcats\nwikitext\ntext/x-wiki\n#REDIRECT [[Computer accessibility]]\n\n{{R from move}}\n{{R from CamelCase}}\n{{R unprintworthy}}\n42l0cvblwtb4nnupxm6wo000d27t6kf\n\n\n',
 str)

In [14]:
data = []
for page in pages:
    row = {
        'title': get_title_from_page_xml(page),
        'text': get_text_from_page_xml(page),
    }
    data.append(row)

In [15]:
df = pd.DataFrame(data)
df.head()

,text,title
0,\nAccessibleComputing\n0\n10\n\n\n854851586\n8...,AccessibleComputing
1,\nAnarchism\n0\n12\n\n893307238\n893261087\n20...,Anarchism
2,\nAfghanistanHistory\n0\n13\n\n\n783865149\n74...,AfghanistanHistory
3,\nAfghanistanGeography\n0\n14\n\n\n783865160\n...,AfghanistanGeography
4,\nAfghanistanPeople\n0\n15\n\n\n783865293\n616...,AfghanistanPeople


### Challenge

#### 1. Create a Pandas dataframe containing the title and text of each page.

* Implement the `get_text_from_page_xml` function above.
* Re-create the dataframe with the text field filled in.

#### 2. Identify the five pages that have the _longest_ text.

* Find the length of each page's `<text>...</text>` element and add it to your dataframe.
* Sort the data frame by text length, descending.
* What are the titles of the five longest articles?

In [16]:
text_length = []
for row in range(len(df.text)):
    text_length.append(len(df.text[row]))

In [27]:
df['text_length'] = text_length

In [28]:
df.columns

Index(['text', 'title', 'text_length'], dtype='object')

In [30]:
df.sort_values('text_length', ascending=False).head()

,text,title,text_length
10165,\nList of compositions by Johann Sebastian Bac...,List of compositions by Johann Sebastian Bach,433376
15129,\nPakistan\n0\n23235\n\n892344051\n892318762\n...,Pakistan,377146
15287,\nPhilippines\n0\n23440\n\n893390996\n89338606...,Philippines,358169
9299,\nForeign relations of India\n0\n14604\n\n8908...,Foreign relations of India,317905
8838,\nHistory of India\n0\n13890\n\n892803276\n892...,History of India,316989
